# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read CSV
data_file_path = "../WeatherPy/weather_data.csv"

# Create a variable for DF
weather_df = pd.read_csv(data_file_path)
weather_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.65,115.33,46.99,89,74,4.85,AU,1596057156
1,Mandlā,22.60,80.38,76.35,88,100,2.55,IN,1596057415
2,Upernavik,72.79,-56.15,43.18,88,59,4.68,GL,1596057342
3,Kapaa,22.08,-159.32,79.00,78,90,16.11,US,1596057415
4,Olafsvik,64.89,-23.71,51.84,79,79,2.91,IS,1596057568


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)

# Plotting Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Creating new DF based on weather criteria
wind_spd_df = weather_df.loc[weather_df['Wind Speed'] < 10]
cloud_df = wind_spd_df.loc[weather_df['Cloudiness'] < 1]
max_temp_df = cloud_df.loc[weather_df['Max Temp'] < 80]
new_df = max_temp_df.loc[weather_df['Max Temp'] > 70]

new_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Slavonski Brod,45.16,18.02,78.01,81,0,2.48,HR,1596057569
35,Muros,42.78,-9.06,79.00,64,0,3.36,ES,1596057424
56,Guozhen,34.37,107.36,72.45,71,0,2.35,CN,1596057431
65,Vicuña,-30.03,-70.71,73.00,50,0,3.09,CL,1596057434
68,Areosa,41.72,-8.86,78.80,88,0,3.36,PT,1596057321
91,Yashalta,46.34,42.28,75.58,35,0,4.81,RU,1596057441
156,Kabinda,-6.13,24.48,72.32,28,0,1.12,CD,1596057461
267,Carutapera,-1.20,-46.02,78.28,85,0,4.07,BR,1596057490
279,Caravelas,-17.71,-39.25,70.54,72,0,4.79,BR,1596057493
309,Luan,46.36,6.98,75.20,60,0,1.12,CH,1596057488


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Taking the only columns I need
hotels_df = new_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Adding 'Hotel Name' column
hotels_df['Hotel Name'] = ''

In [10]:
# Creating empty list for the closest hotels
hotels = []

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotels_df.iterrows():
    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_req = requests.get(base_url, params=params)

    # convert to json
    hotel_res = hotel_req.json()
    
    # print(json.dumps(hotel_res, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotels.append(hotel_res["results"][0]["name"])
        
    except IndexError:
        hotels.append(np.nan)

# Adding search results to DF
hotels_df['Hotel Name']= hotels
hotels_df.dropna()


,City,Country,Latitude,Longitude,Closest Hotel,Hotel Name
10,Slavonski Brod,HR,45.16,18.02,Hotel Savus,Art Hotel
35,Muros,ES,42.78,-9.06,Hotel La Muradana,A Casa da Fonte
56,Guozhen,CN,34.37,107.36,Lily Garden Hotel,Sunshine Business Hotel
65,Vicuña,CL,-30.03,-70.71,Rucka-Inn Cottages & Domes,Cabañas La Villa Sueca
68,Areosa,PT,41.72,-8.86,Hotel Flôr de Sal,Pousada Viana do Castelo
267,Carutapera,BR,-1.20,-46.02,HOTEL R. SOARES,HOTEL R. SOARES
279,Caravelas,BR,-17.71,-39.25,Mar Doce de Abrolhos,Mar Doce de Abrolhos
309,Luan,CH,46.36,6.98,Hôtel du Port,Hôtel du Port
336,Zatoka,UA,46.07,30.47,Ruta,Ruta
392,Kikuchi,JP,32.98,130.82,旅館宝来館,里山の隠れ宿 花富亭（かふうてい）


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))